## 测试和存储分类器
到目前为止，已经完成从原始数据集中创建决策树，并使用 Matplotlib 库绘制树形图。现在，我们将把重点转移到如何利用决策树执行数据分类。

### 测试算法：使用决策树执行分类
依靠训练数据构造决策树之后，可以将其用于实际数据的分类。在执行数据分类时，需要使用决策树以及用于构造决策树的标签向量。然后，程序比较测试数据与决策树上的数值，递归执行该过程直到进入叶子节点；最后将测试数据定义为叶子节点所属的类型。

In [1]:
def classify(input_tree, feat_labels, test_vec):
    first_str = list(input_tree.keys())[0]
    second_dict = input_tree[first_str]
    feat_index = feat_labels.index(first_str)
    for key in second_dict.keys():
        if test_vec[feat_index] == key:
            if type(second_dict[key]).__name__ == 'dict':
                class_label = classify(second_dict[key], feat_labels, test_vec)
            else:
                class_label = second_dict[key]
    return class_label

【问题】：存储带有特征的数据时，程序无法确定特征在数据集中的位置，例如前面例子中的第一个用于划分数据集的特征是 no surfacing 属性，但是在实际数据集中该属性存储在哪个位置？是第一个属性还是第二个属性？特征标签列表将帮助程序处理这个问题。使用 index() 方法查找当前列表中第一个匹配 first_str 变量的元素。然后递归遍历整棵树，比较 test_vec 变量中的值与树节点的值，如果到达叶子节点，则返回当前节点的分类标签。

In [2]:
def create_dataset():
    dataset = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no']
    ]
    labels = ['no surfacing', 'flippers']
    return dataset, labels

In [4]:
my_dat, labels = create_dataset()

In [10]:
labels

['no surfacing', 'flippers']

In [11]:
def retrieve_tree(i):
    list_of_trees = [
        {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}},
        {'no surfacing': {0: 'no', 1: {'flippers': {0: {'head': {0: 'no', 1: 'yes'}}, 1: 'no'}}}}
    ]
    return list_of_trees[i]

In [12]:
my_tree = retrieve_tree(0)

In [13]:
classify(my_tree, labels, [1, 0])

'no'

In [14]:
classify(my_tree, labels, [1, 1])

'yes'

### 使用算法：决策树的存储
构造决策树是很耗时的任务，即使处理很小的数据集。然而用创建好的决策树解决分类问题，则可以很快完成。因此，为了节省计算时间，最好能够在每次执行分类时调用已经构造好的决策树。

为了解决这个问题，需要使用 Python 模块 pickle 序列化对象。序列化对象可以在磁盘上保存对象，并在需要的时候偶读取出来。任何对象都可以执行序列化操作，字典对象也不例外。

In [31]:
def store_tree(input_tree, filename):
    import pickle
    with open(filename, 'wb') as file:
        pickle.dump(input_tree, file)

In [32]:
def grab_tree(filename):
    import pickle
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [33]:
store_tree(my_tree, 'classifierStorage.txt')

In [34]:
grab_tree('classifierStorage.txt')

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

通过上面的代码，我们可以将分类器存储在磁盘上，而不用每次对数据分类时重新学习一遍，这也是决策树的优点之一，像之前所学的 K-近邻算法就无法持久化分类器。